## concrete valueを最初から利用するver.

In [173]:
import sympy as sm
import numpy as np
import itertools

In [174]:
sm.init_printing()

In [175]:
def g(x):
    return sm.tanh(x)

def dg(x):
    return 1 - sm.tanh(x) * sm.tanh(x)

def ddg(x):
    return 2 * (sm.tanh(x) * sm.tanh(x) - 1) * sm.tanh(x)

In [184]:
# https://stackoverflow.com/questions/22857162/multivariate-taylor-approximation-in-sympy

def mv_series(function_expression, 
              variable_list, 
              evaluation_point, 
              degree):
    
    n_var = len(variable_list)
    
    # list of tuples with variables and their evaluation_point coordinates, ready for the subs() method
    point_coordinates = [(i, j) for i, j in (zip(variable_list, evaluation_point))]
    
    # list with exponents of the partial derivatives
    deriv_orders = list(itertools.product(range(degree + 1), repeat=n_var))
    
    # Discarding some higher-order terms
    deriv_orders = [deriv_orders[i] for i in range(len(deriv_orders)) if sum(deriv_orders[i]) <= degree]
    n_terms = len(deriv_orders)
    
    # same as before, but now ready for the diff() method
    deriv_orders_as_input = [list(sum(list(zip(variable_list, deriv_orders[i])), ())) for i in range(n_terms)]
    
    polynomial = 0
    for i in range(n_terms):
        partial_derivatives_at_point = function_expression.diff(*deriv_orders_as_input[i]).subs(point_coordinates)
        # e.g. df/(dx*dy**2) evaluated at (x0,y0)
        
        denominator = sm.prod([sm.factorial(j) for j in deriv_orders[i]])
        # e.g. (1! * 2!)
        
        distances_powered = sm.prod([(sm.Matrix(variable_list) - sm.Matrix(evaluation_point))[j] ** deriv_orders[i][j] for j in range(n_var)])  
        # e.g. (x-x0)*(y-y0)**2
        
        polynomial += partial_derivatives_at_point / denominator * distances_powered
    return polynomial


In [185]:
x_dim = 3
r_dim = 4
#r_dim = 64
#r_dim = 100


In [186]:
# Symbols
x       = sm.MatrixSymbol('x', x_dim, 1)
x_dot   = sm.MatrixSymbol('\dot{x}', x_dim, 1)

In [187]:
# Concrete values
def calc_scale_with_spectral_radius(mat, rho):
    cur_spectral_radius = max(abs(np.linalg.eigvals(mat)))
    return rho / cur_spectral_radius

rng = np.random.RandomState(0)

r_s_c = sm.Matrix(rng.uniform(low=-0.5, high=0.5, size=(r_dim, 1)))

x_s_c = sm.Matrix(np.zeros((x_dim, 1)))

B_scale = 0.1
B_np = rng.uniform(low=-1.0, high=1.0, size=(r_dim, x_dim)) * B_scale

B_c = sm.Matrix(B_np)

A_np = rng.uniform(low=-1.0, high=1.0, size=(r_dim, r_dim))
sparsify = np.vectorize(lambda v: v if rng.random() < 0.05 else 0.0)
A_np = sparsify(A_np)

rho = 0.01
A_scale = calc_scale_with_spectral_radius(A_np, rho)
print(f'A_scale={A_scale}')
#A_np = A_np * A_scale

# TODO: 最適化可能

A_c = sm.Matrix(A_np)

d_c       = sm.Matrix(np.zeros((r_dim, 1)))
d_s_c     = A_c * r_s_c + d_c

ArBxd_c = A_c * r_s_c + B_c * x_s_c + d_s_c
# 要素積のためにshapeを合わせておく
ArBxd_c_rep = sm.Matrix.hstack(*[ArBxd_c for _ in range(r_dim)])

A_s_c = sm.hadamard_product(ArBxd_c_rep.applyfunc(dg), A_c) - sm.eye(r_dim)
A_s_inv_c = A_s_c ** -1

gamma_c = 100

A_scale=inf


/var/folders/g8/lw05g79s3yv331w4vf305fkh0000gn/T/ipykernel_48825/1373796577.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  return rho / cur_spectral_radius


In [188]:
B       = B_c
d_s     = d_s_c
r_s     = r_s_c
A_s_inv = A_s_inv_c
A       = A_c
gamma   = gamma_c

In [189]:
expr0 = sm.hadamard_product((B * x + d_s).applyfunc(dg), A * r_s)
expr1 = (B * x + d_s).applyfunc(g)
expr2 = sm.hadamard_product((B * x + d_s).applyfunc(ddg), A * r_s)
expr3 = (B * x + d_s).applyfunc(dg)

expr2_3 = sm.hadamard_product(expr2 - expr3, B * x_dot)

expr = A_s_inv * (expr0 - expr1) + (1 / gamma) * (A_s_inv ** 2) * expr2_3

xs = [x[i,0] for i in range(x_dim)]
x_dots = [x_dot[i,0] for i in range(x_dim)]

x_zero = [0] * x_dim
x_dot_zero = [0] * x_dim

degree = 2

In [190]:
x_dots

In [191]:
%%time

coeff_dicts = []

for r_index in range(r_dim):
    r_ex = mv_series(expr[r_index,0], 
                     xs + x_dots, 
                     x_zero + x_dot_zero, degree)
              
    p_c = sm.poly(r_ex, xs + x_dots).as_poly()
    coeff_dicts.append(p_c.as_dict())
    
    print(r_index)
    #..
    #break
    #..

0
1
2
3
CPU times: user 1.49 s, sys: 6.9 ms, total: 1.5 s
Wall time: 1.49 s


In [192]:
coeff_dicts

In [193]:
# {(0, 0, 0, 0, 0, 0):0.0019240475738571